# ROS параметры

Есть еще один аспект, который называется [__сервер параметров__](http://wiki.ros.org/Parameter%20Server).

__Параметрами__ в ROS называются просто данные, которые хранятся под определенными именами и пространствами имен. Как было рассмотрено ранее, запуск узла в пространстве имен меняет конечное имя узла, а также топика. Аналогично с этим, вся работа узла с параметрами (чтение, запись) происходит в том пространстве имен, которому он принадлежит.

> Сервер параметров хранит параметры и привязан к мастеру. Перезапуск мастера приводит к потере всех ранее заданных параметров.

Пора знакомиться с основной утилитой работы с параметрами =)

In [ ]:
%%bash
rosparam -h

Теперь попробуем проверить список параметров в системе

In [ ]:
%%bash
rosparam list

Давайте поработаем с параметром `/rosdistro`

In [ ]:
%%bash
rosparam get /rosdistro

Ожидаемый вывод, не так ли?)

А теперь попробуем задать свой параметр и сразу прочитать его

In [ ]:
%%bash
rosparam set /my_param 'Hello =)'
rosparam set /my_set '{ 'P': 10.0, 'I': 1.0, 'D' : 0.1 }'

rosparam get /my_param
rosparam get /my_set
rosparam get /my_set/P

Вроде все логично =) А теперь попробуйте перезапустить ячейку с выводом списка параметров в системе.

Как видно из вывода хелпа, параметрами также можно управлять, удаляя их, также выгружать в файл и загружать из файла.

## ROS Python параметры

Теперь рассмотрим применение параметров внутри узлов. Использование Jupyter notebook позволяет писать части узлов и выполнять их без создания отдельных файлов. Опираться рекомендуется на [страницу из туториала про параметры](http://wiki.ros.org/rospy_tutorials/Tutorials/Parameters).

Для начала стандартный и знакомый для Python узла код. Что интересно, в Python для работы с параметрами не обязательно регистрировать узел, тем не менее мы это сделаем на ряду с импортом основного модуля. Для пущего интереса зададим 

In [ ]:
import rospy

rospy.init_node('params_study')

> Не забудьте выполнить ячейку! Как только вы это сделаете, у вас будет подключен модуль `rospy` и действия будут происходить от узла `params_study` для всех ячеек, которые вы выполните далее. (Работает до перезапуска ядра Jupyter - кнопка в панели управления)

Ну и начнем рассматривать, что же можно сделать с параметрами в `rospy`?  
Рассмотрим основные типы обращений к параметрам:

In [ ]:
# Обращение глобально - как видно в начале стоит `/`
# Не обращаем внимания на ns, ищем именно такой путь параметра и никак иначе
distro = rospy.get_param('/rosdistro')
print(distro)

# Обращение локально - в начале не стоит `/`
# Допустим мы запустили узел, указав ns:=my_ns
# Тогда вызов данной функции будет пытаться найти парметр по пути - `/my_ns/my_set` 
my_set_param = rospy.get_param('my_set')
print(my_set_param)

Есть еще такой тип - приватные параметры. Они отличаются тем, что они привязаны к узлу, тем не менее всей системе они также доступны. Лучше объясню на примере:

In [ ]:
# Зададим параметры из узла, локальный, глобальный и приватный
rospy.set_param('~ros_priv_param', 'Hi, I am private =)')
rospy.set_param('ros_loc_param', 'Hi, I am local =)')
rospy.set_param('/ros_glob_param', 'Hi, I am global =)')

In [ ]:
%%bash
# Теперь проверим список параметров в системе
rosparam list

По результатам, вы, наверное, видите в списке наши заданные параметры:
```
/params_study/ros_priv_param
/ros_glob_param
/ros_loc_param
```

Все получилось! Между глобальным параметром и локальным разницы нет - по-умолчанию у ROS ns равен `/`. К сожалению из Jupyter не нашел способа задать ns =(. А вот приватный отличается тем, что в его префиксе присутствует имя узла. Это говорит о том, что у нему можно добраться любому узлу, но концептуально он сидит на конкретном узле. Соответственно, можно запустить много одинаковых узлов (например с помощью флага анонимности) и получить такое же количество параметров. Много букаф =)
> Задачка - с помощью утилиты, рассмотренной ранее, проверьте значения заданных параметров `ros_priv_param`, `ros_glob_param`, `ros_loc_param`

Значит так, мы научились получать параметры от сервера параметров, задавать (если их не существовало - создавать, функция все равно одна и та же). Еще один момент. Бывает такое, хотим работать с параметром, а его нету в сервере (причины могут быть разные). В этом случае при запросе происходит это:

In [ ]:
not_exist_param = rospy.get_param('i_do_not_exist')
print(not_exist_param)

Огромный ERROR, ааа!  
Не паникуем, сейчас будет финт ушами: для начала решим проблему по пути Python - ловим все исключения (ошибки) и после обрабатываем:

In [ ]:
try:
    not_exist_param = rospy.get_param('i_do_not_exist')
except:
    # Not exist or any kind of other problem - set default
    not_exist_param = 'Okay, now default time =0'
    
print(not_exist_param)

А еще можно задать значение по-умолчанию прямо в вызов вторым аргументом:

In [ ]:
not_exist_param = rospy.get_param('i_do_not_exist', 'Okay, now default time =)')
print(not_exist_param)

И все работает! По факту Python трюк проделывает внутри себя эта функция, если мы задаем значение по-умолчанию =)  
Вот так мы научились еще и обрабатывать получение значения по-умолчанию. Есть еще функционал из туториала, удаление параметра, проверка существования параметра и получение списка. Зачем это может понадобиться решайте сами.

In [ ]:
param_name_2_delete = 'ros_glob_param'

# Проверим список параметров, только уже через Python
param_list = rospy.get_param_names()
print(param_list)

# Наличие можно проверить через функционал ROS    
if rospy.has_param(param_name_2_delete):
    print('[ROSWay] Parameter exist')
else:
    print('[ROSWay] Parameter not exist')
    
# И с проверкой удаляем его
if rospy.has_param(param_name_2_delete):
    rospy.delete_param(param_name_2_delete)
    
# Еще раз проверим:
if rospy.has_param(param_name_2_delete):
    print('[ROSWay] Parameter exist')
else:
    print('[ROSWay] Parameter not exist')

Если вывод похож на
```
[ROSWay] Parameter exist
[ROSWay] Parameter not exist
```
значит все окей! До вызова удаления параметр существовал, после - нет =)

Вообще, при желании глянуть на офф доки - можно воспользоваться страницей [API rospy](http://docs.ros.org/api/rospy/html/).

## Управление параметрами

Данная тема может в будущем немного подрасти, сейчас просто хотелось бы обратить небольшое внимание на приватные параметры с точки зрения практики. Обычно узлы стартуют с помощью launch-файлов, поэтому задаются параметры внутри с помощью тэгов `<param>`. Пример из одного из файлов планера:
```xml
    <param name="base_global_planner" value="global_planner/GlobalPlanner" />
    <param name="planner_frequency" value="1.0" />
    <param name="planner_patience" value="5.0" />
```

Еще немного для понимания, пример из драйвера камеры:
```xml
<node ns="stereo" name="left_camera" pkg="usb_cam" type="usb_cam_node" output="screen" >
    <param name="video_device" value="/dev/video0" />
	<param name="image_width" value="640" />
	<param name="image_height" value="480" />
```

Здесь с помощью параметров задается путь девайса (конкретный, так как таких может быть много) и размеры выходного изображения.
> Задание для рассуждения - обсудите с коллегами начинку тэга <node> в примере драйвера камеры.

### Сохранение и загрузка параметров

Так как при закрытии мастера параметры теряются, было бы неплохо узнать, а как сохранять параметры в файл? А как загружать из файла? Такой функционал есть и утилита все та же:

In [ ]:
%%bash
rosparam dump --help

In [ ]:
%%bash
rosparam load --help

In [ ]:
%%bash
# Сохраним все параметры в файл (обычно расширение '.yaml') начиная с пространства имен '/' - то есть все параметры
# Флаг -v для визуального контроля
rosparam dump -v '/tmp/my_dump.yaml' '/'

In [ ]:
%%bash
# Теперь загрузим параметры, но в новое пространство имен
rosparam load -v '/tmp/my_dump.yaml' '/my_new_ns_just_to_make_it_long_for_control_=)'

In [ ]:
%%bash
# Взглянем на список параметров
rosparam list

Как видим, у нас появилась полная копия параметров, только в новом пространстве имен. Еще немного практики для понимания пространства имен

In [ ]:
%%bash
# Сохраним параметры только из пространства /my_set
rosparam dump -v '/tmp/my_dump_special_ns.yaml' '/my_set'

In [ ]:
%%bash
# Загрузим их в новое пространство для пущего примера
rosparam load -v '/tmp/my_dump_special_ns.yaml' '/new_ns_for_special'

In [ ]:
%%bash
# Взглянем на список параметров
rosparam list

А теперь мозговой штурм! На этом моменте можно очень хорошо понять принцип пространства имен:
На это можно смотреть как на систему папок. Если мы указываем для сохранения конкретное пространство, то все, что лежит внутри ns (далее за `/` этой папки) будет сохранено. При загрузке, мы указываем папку, с которой начать запись. 
> TODO -> может больше расписать, на этом моменте желательно, чтобы люди до конца разобрались, как работать с ns!

Думаю, объяснять функционал `delete` сильно не стоит:

In [ ]:
%%bash
# Сносим целое пространство и смотрим, что получилось
rosparam delete -v '/my_new_ns_just_to_make_it_long_for_control_=)'

rosparam list

Вот и подчистили наш сервер параметров =)  
Теперь к практическим навыкам - вспомним, что запуск launch-файла запускает также и мастера, если тот ранее не был запущен. А сервер параметров завязан на мастера. Значит может понадобиться функционал загрузки параметров на момент запуска узлов - есть он у меня для вас =)  
```xml
<rosparam file="config/costmap_common.yaml" command="load" ns="global_costmap" />
```

В этом примере показан тэг `<rosparam>` и его параметры. На самом деле, параметры схожи с опциями утилиты:
- file - файл с сохраненными параметрами;
- command - может быть [load / dump / delete];
- ns - пространство имен, куда загрузить / откуда сохранить / что удалить.

Таким образом можно достаточно гибко управлять параметрами ваших узлов. Еще один момент, давайте посмотрим на содержимое файла с параметрами, он достаточно понятный и функционал Jupyter это позволяет:

In [ ]:
%cat '/tmp/my_dump.yaml'

Формат <имя параметра> : <значение> - это специальный формат файлов `YAML`. Для массивов и вложенных параметров происходит обертка вложенности скобками `{}` и внутри по идентичному принципу.

## Параметры в C++

Чтож, во-первых, предлагаю посмотреть [туториал на офф странице](http://wiki.ros.org/roscpp_tutorials/Tutorials/Parameters). Во-вторых, принцип работы аналогичный, поэтому сильно расписывать и рассматривать не будем, обсудим только базовый функционал.

> Весь дальнейший функционал принадлежит классу `NodeHandle` и требует созданного объекта этого класса. Далее полагаем, что где-то вызвали `ros::NodeHandle n;` Вот вам еще [API NodeHandle](http://docs.ros.org/kinetic/api/roscpp/html/classros_1_1NodeHandle.html).

```cpp
bool getParam (const std::string& key, parameter_type& output_value) const;
```

Функция получает значение параметра по переданному ключу (имени параметра). `parameter_type` соответствует типу значения параметра (string, bool, int, double). Есть еще специальный тип `XmlRpcValue`, но его рассморение не та важно и может быть рассмотрено из [документации](http://docs.ros.org/kinetic/api/xmlrpcpp/html/classXmlRpc_1_1XmlRpcValue.html). Возвращает функция `true`, если параметр был удачно получен, и `false`, если параметр не удалось получить (не существует, например). Пример получения параметра:
```cpp
    std::string s;
    if (n.getParam("my_param", s))
    {
      ROS_INFO("Got param: %s", s.c_str());
    }
    else
    {
      ROS_ERROR("Failed to get param 'my_param'");
    }
```

В строку `s` будет записано значение параметра `my_param`, если таковой существует. Иначе будет выполнен блок `else`.

```cpp
n.setParam("my_param", "hello there");
```

Функция в примере устанавливает параметр. Первый агрумент - имя параметра, второй - значение. Значение может быть типами string, bool, int, double или XmlRpcValue.

```cpp
if ( n.hasParam("my_param") )
{
    ROS_INFO("Delete param named 'my_param'");
    n.deleteParam("my_param");
}
```

Пример на проверку наличия параметра на сервере и удаления. Функции достаточно говорящие =)

## В результате

- Мы познакомились с сервером параметров и утилитой работы с параметрами.
- Научились пользовать параметры в Python и С++.
- Рассмотрели практические применения утилит и параметров в rolaunch.